# Main Pipleline

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

Cameras needed to be calibrated first to remove distortion

In [2]:
import calibrate_camera # calibration module
mtx, dist = calibrate_camera.calibrate(9, 6, 'camera_cal/*.jpg')

In [7]:
mtx, dist

(array([[1.15777818e+03, 0.00000000e+00, 6.67113857e+02],
        [0.00000000e+00, 1.15282217e+03, 3.86124583e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[-0.24688507, -0.02373156, -0.00109831,  0.00035107, -0.00259866]]))

after we got the matrix and distortion coefficients we apply them to checkboards images to check the results

In [4]:
import glob
import cv2
images_names = glob.glob('camera_cal/*.jpg')
for index, image_name in enumerate(images_names):
    image = cv2.imread(image_name)
    image_size = (image.shape[1], image.shape[0])
    undistored_image = calibrate_camera.undistort(image, mtx, dist)
    cv2.imwrite('output_images/calibrated_boards/test{}.jpg'.format(index), undistored_image)

we save the results in a pickle dictionary so we don't need to repeat the pipeline and gain performance improvements

In [ ]:
import pickle
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ) )

In [6]:
images_names = glob.glob('test_images/*.jpg')
for index, image_name in enumerate(images_names):
    image = cv2.imread(image_name)
    image_size = (image.shape[1], image.shape[0])
    undistorted_image = calibrate_camera.undistort(image, mtx, dist)
    cv2.imwrite('output_images/calibrated_roads/road{}.jpg'.format(index), undistorted_image)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import sobel # sobel module reads images in RGB using matplot lib
images_names = glob.glob('output_images/calibrated_roads/*.jpg')
ksize = 3
for index, image_name in enumerate(images_names):
    image = mpimg.imread(image_name)
    combined_binary = sobel.get_binary(image, ksize)
    plt.imsave('output_images/roads_binary/{}.jpg'.format(index), combined_binary,
               cmap='gray')

ModuleNotFoundError: No module named 'matplotlib'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import bird_view
import lanes
import cv2
import rad

images_names = glob.glob('output_images/roads_binary/*.jpg')
undists = glob.glob('output_images/calibrated_roads/*.jpg')
for index, image_name in enumerate(images_names):
    image = mpimg.imread(image_name)
    binary_warped, matrix, matrix_inv = bird_view.get_bird_view(image)
    plt.imsave('output_images/roads_view/{}.jpg'.format(index), binary_warped,
               cmap='gray')
    
    out_img, left_fitx, right_fitx, ploty = lanes.fit_polynomial(binary_warped)
    undist = plt.imread(undists[index])
    result = lanes.draw_path(binary_warped, left_fitx, right_fitx, ploty, matrix_inv, undist)
    
    left_curverad, right_curverad = rad.measure_curvature_real(left_fitx, right_fitx, ploty)
    print(left_curverad, 'm', right_curverad, 'm')
    plt.imsave('output_images/roads_maped/{}.jpg'.format(index), result)

1703.6446855675265 m 2135.644737499449 m
872.8047419398854 m 15744.56429862752 m
1143.2132375411938 m 3582.884890838442 m
730.4273313766162 m 385.45567005823216 m
1033.7684975481518 m 1104.0563685802676 m
8601.694652626713 m 464.26109146783597 m
1001.0744467532164 m 497.5813085092501 m
1015.241119987217 m 924.1995423303791 m
